In [18]:
import cv2
import numpy as np
import random as rng
import time
'''Change the parameters to conduct a real time detection'''
CANNY_THRESHOLD = 25
MEDIAN_BLUR_K_SIZE = 9
MORPH_K_SIZE = 1

center_list = {} 
frame_no =1
frame_rate = []

time_rate = []

srcPiCam = 'output2jd.mp4'
pcap = cv2.VideoCapture(srcPiCam)
Total_frames = int(pcap.get(cv2.CAP_PROP_FRAME_COUNT))
if pcap.isOpened():
        print(f'Pupil camera available:')
        
        
kernel = np.ones((MORPH_K_SIZE, MORPH_K_SIZE), np.uint8)

while True:
    
    pret, pframe = pcap.read()
    if pret: 
        start = time.perf_counter()
        pframe = pframe[0:350, 0:500]
        output = pframe.copy()
        src_gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
        src_gray = cv2.medianBlur(src_gray, MEDIAN_BLUR_K_SIZE)
        opening = cv2.morphologyEx(src_gray, cv2.MORPH_OPEN, kernel)
        canny = cv2.Canny(opening, CANNY_THRESHOLD, CANNY_THRESHOLD * 2)
        _contours, _ = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        _contours_filtered = []
        #print("Initial Contours : {}".format(len(_contours)))
        #print("Filtered Contours:")
        for i, c in enumerate(_contours):
            try:
                convex_hull = cv2.convexHull(c)
                area_hull = cv2.contourArea(convex_hull)
                # print("{} area convex hull {}".format(i, area_hull))
                if 600 < area_hull:  # filtering based on area
                    circumference_hull = cv2.arcLength(convex_hull, True)
                    circularity_hull = (4 * np.pi * area_hull) / circumference_hull ** 2
                    if 0.8 < circularity_hull:  # filtering based on circularity
                        #print("convex hull :{} Circularity :{} Area : {}".format(i, circularity_hull, area_hull))
                        _contours_filtered.append(convex_hull)
            except ZeroDivisionError:
                print("Division by zero for contour {}".format(i))
        
        _drawing = output
        minEllipse = [None] * len(_contours_filtered)
        for i, c in enumerate(_contours_filtered):
            color = (rng.randint(0, 256), rng.randint(0, 256), rng.randint(0, 256))
            minEllipse[i] = cv2.fitEllipse(c)
            cv2.drawContours(_drawing, _contours_filtered, i, color)
            (x, y), (MA, ma), angle = minEllipse[i]
            center = (x, y)
            area_contour_hull = cv2.contourArea(c)
            area_ellipse = (np.pi / 4) * MA * ma
            #print("Area Ellipse :{} Area Contour Hull :{}".format(area_ellipse, area_contour_hull))
            center_list.update({frame_no: center})
            cv2.ellipse(_drawing, minEllipse[i], color=color, thickness=2)
            
        
    end = time.perf_counter()
    dtime = (end-start)
    if dtime > 0:  # To avoid division by zero
        fps = 1 / dtime
    else:
        fps = float('inf')  # If dtime is 0, processing was instantaneous
    cv2.putText(_drawing, str(fps), (0, 300),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.imshow('dframe',_drawing)
    
    time_rate.append(dtime)
    frame_rate.append(fps)
    frame_no += 1
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

pcap.release()
cv2.destroyAllWindows()

Pupil camera available:


In [11]:
print(len(center_list))

1310


In [12]:
avg = sum(frame_rate) / len(frame_rate)
print(avg)

139.034149071131


In [13]:
srcPiCam = 'output3aa.mp4'
pcap = cv2.VideoCapture(srcPiCam)
Total_frames = int(pcap.get(cv2.CAP_PROP_FRAME_COUNT))

In [14]:
data_file = "jd_rectangle.txt"
##### CONVERTING ANNOTATED DATA IN DICTIONARY
rectangles_dict = {}
with open(data_file, "r") as file:
    for line in file:
        # Split the line into frame number and coordinates
        frame_num, *coord_str = line.strip().split(":")
        
        # Convert frame number to integer
        frame_num = int(frame_num)
        
        coordinates = []
        for coord in coord_str:
            # print(coord)
            x1y1,x2y2 = coord.split(";")
            # split coordinates into its individual components
            x1,y1 = x1y1.strip("()").split(",")
            x2,y2 = x2y2.strip("()").split(",")
            
        rectangles_dict[frame_num] = [(int(x1), int(y1)) , (int(x2),int(y2))]

# Print the dictionary
print(rectangles_dict)



{1: [(263, 48), (370, 129)], 2: [(235, 69), (338, 149)], 3: [(243, 78), (334, 153)], 4: [(236, 86), (323, 162)], 5: [(230, 90), (326, 167)], 6: [(233, 89), (323, 164)], 7: [(231, 89), (325, 166)], 8: [(235, 90), (319, 166)], 9: [(227, 90), (319, 164)], 10: [(234, 86), (326, 161)], 11: [(247, 77), (341, 145)], 12: [(257, 59), (342, 142)], 13: [(253, 64), (330, 124)], 14: [(250, 58), (346, 139)], 15: [(252, 57), (336, 143)], 16: [(252, 59), (346, 136)], 17: [(241, 56), (340, 143)], 18: [(248, 55), (344, 146)], 19: [(249, 59), (338, 131)], 20: [(250, 58), (338, 133)], 21: [(257, 58), (334, 134)], 22: [(255, 58), (338, 135)], 23: [(252, 54), (336, 137)], 24: [(254, 57), (330, 125)], 25: [(273, 59), (349, 126)], 26: [(271, 58), (353, 131)], 27: [(275, 56), (349, 132)], 28: [(257, 51), (349, 135)], 29: [(276, 52), (343, 125)], 30: [(273, 54), (342, 128)], 31: [(275, 56), (337, 129)], 32: [(273, 53), (341, 125)], 33: [(275, 52), (348, 134)], 34: [(274, 53), (344, 127)], 35: [(271, 55), (349, 

In [15]:
detection = []
for key in center_list.keys():
    if key in rectangles_dict.keys():
        
        center_value = center_list[key]
        rectangle_values = rectangles_dict[key]
        x_min, y_min = rectangle_values[0]
        x_max, y_max = rectangle_values[1]
        
        # check if pupil center lies in bounding rectangle
        if x_min <= center_value[0] <= x_max and y_min <= center_value[1] <= y_max:
            detection.append(1)
            print(f"Key {key}: Center value {center_value} is within the rectangle.")
        else:
            detection.append(0)
            print(f"Key {key}: Center value {center_value} is outside the rectangle.")
    else:
        print(f"Key {key} is not present in rectangles_dict.")
print(detection)

Key 1: Center value (309.5375061035156, 82.74885559082031) is within the rectangle.
Key 2: Center value (286.9898376464844, 106.09342956542969) is within the rectangle.
Key 4: Center value (277.11187744140625, 122.05174255371094) is within the rectangle.
Key 5: Center value (276.93865966796875, 122.94929504394531) is within the rectangle.
Key 6: Center value (277.10546875, 123.240234375) is within the rectangle.
Key 7: Center value (277.2193603515625, 123.2043228149414) is within the rectangle.
Key 8: Center value (277.1532287597656, 123.25330352783203) is within the rectangle.
Key 9: Center value (277.11712646484375, 123.35256958007812) is within the rectangle.
Key 10: Center value (277.6808166503906, 122.69391632080078) is within the rectangle.
Key 13: Center value (293.7033996582031, 94.23794555664062) is within the rectangle.
Key 14: Center value (293.6053771972656, 94.229736328125) is within the rectangle.
Key 15: Center value (293.58404541015625, 94.27493286132812) is within the 

In [16]:
Total_no_frames = Total_frames
Correctly_detect_pupil = detection.count(1) # Correctly detected pupil frames
Incorrect_detect_pupil = detection.count(0) # Incorrectly detected pupil frames
Missed_detect = Total_no_frames- Correctly_detect_pupil-Incorrect_detect_pupil # No detection of pupil in frames
print(f"Total frames = {Total_no_frames}")
print(f"Correctly detected pupil frames = {Correctly_detect_pupil}")
print(f"Incorrectly detected pupil frames = {Incorrect_detect_pupil}")
print(f"No detection of pupil in frame = {Missed_detect}")

Total frames = 1412
Correctly detected pupil frames = 1279
Incorrectly detected pupil frames = 31
No detection of pupil in frame = 102


In [17]:
Accuracy = (Correctly_detect_pupil/Total_no_frames)*100
Incorrect_detect_rate = (Incorrect_detect_pupil/Total_no_frames)*100
Missed_detect_rate = (Missed_detect/Total_no_frames)*100
Precision = (Correctly_detect_pupil/(Correctly_detect_pupil+Incorrect_detect_pupil))*100
Recall = (Correctly_detect_pupil/(Correctly_detect_pupil+Missed_detect))*100
F1_score = 2*(Precision*Recall)/(Precision+Recall)

print(f"Accuracy: {Accuracy:.2f}%")
print(f"Incorrect Detection Rate: {Incorrect_detect_rate:.2f}%")
print(f"Missed Detection Rate: {Missed_detect_rate:.2f}%")
print(f"Precision: {Precision:.2f}%")
print(f"Recall: {Recall:.2f}%")
print(f"F1 Score: {F1_score:.2f}")

Accuracy: 90.58%
Incorrect Detection Rate: 2.20%
Missed Detection Rate: 7.22%
Precision: 97.63%
Recall: 92.61%
F1 Score: 95.06
